# Соревнование - https://www.kaggle.com/c/departure-delay2

# Описание данных
1. Unnamed: 0 - лишний безинформативный столбец
2. Year - год
3. Month - месяц
4. DayOfWeek - день недели
5. DepTime - время отправления в формате HH:MM (надо преобразовать)
6. CRSDepTime - запланированное время отправления в формате HH:MM (надо преобразовать)
7. ArrTime - время вылета в формате HH:MM (надо преобразовать)
8. CRSArrTime - запланированное время прибытия в формате HH:MM (надо преобразовать)
9. UniqueCarrier - ID оператора связи (???)
10. FlightNum - номер рейса 
11. TailNum - бортовой номер самолета
12. ActualElapsedTime - фактическое время полета в минутах
13. CRSElapsedTime - запланированное время полета в минутах
14. AirTime - время полета в воздухе в минутах
15. ArrDelay - задержка прибытия в минутах
16. Origin - аэропорт отправления
17. Dest - аэропорт назначения
18. Distance - дистанция полета
19. ??
20. ??
21. Cancelled - статус отмены (1 0)
22. CancellationCode - код отмены
23. Diverted - статус переадресации (???)
24. CarrierDelay - задержка в минутах, связанная с оператором связи
25. WeatherDelay - задержка в минутах, связанная с погодными факторами 
26. NASDelay - задержка в минутах, относящаяся к Национальной авиационной системе
27. SecurityDelay - задержка в минутах, связанная с факторами безопасности
28. LateAircraftDelay - задержка в минутах, связанная с опозданием воздушного судна

* Столбец Unnamed: 0 можно удалить сразу, никакой информации он не несет.
* Стоит преобразовать даты: сконкатенировать день, месяц и год. Пока не знаю как их преобразовать для линейной регрессии, удалю.
* Последние столбцы с задержками можно схлопнуть в один total_factor_delay, но в них очень   много пропущенных значений, нужно оптимальным образом их заполнить (сначала попробую заполнить их медианным значением)
* Такие данные как CancellationCode, Diverted, Dest, Origin - вряд ли будут релевантны

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('departure-delay2/x_train.csv')
test = pd.read_csv('departure-delay2/x_test.csv')
target = pd.read_csv('departure-delay2/y_train.csv')


In [ ]:
print('Shape --->',train.shape)
print(train.info(), '\n')
print('ISNA SUM ---->\n',train.isna().sum())
train.head(5)

In [ ]:
train.isna().sum().index

In [ ]:
print('Shape --->',test.shape)
print(test.info())
print('ISNA SUM ---->\n',test.isna().sum())
test.head(5)


In [ ]:
print('Shape --->',target.shape)
print(target.info())
target.head(5)

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train['id'] = [i for i in range(len(train))]
train = train.merge(target, on = 'id')
x_train = train.drop(['Unnamed: 0','UniqueCarrier', 'TailNum', 'Origin', 'Dest', 'CancellationCode'], axis=1)
x_test = test.drop(['Unnamed: 0','UniqueCarrier', 'TailNum', 'Origin', 'Dest', 'CancellationCode'], axis=1)

In [ ]:
x_train['CarrierDelay'].fillna(x_train['CarrierDelay'].median(), inplace = True)
x_train['WeatherDelay'].fillna(x_train['WeatherDelay'].median(), inplace = True)
x_train['NASDelay'].fillna(x_train['NASDelay'].median(), inplace = True)
x_train['SecurityDelay'].fillna(x_train['SecurityDelay'].median(), inplace = True)
x_train['LateAircraftDelay'].fillna(x_train['LateAircraftDelay'].median(), inplace = True)
x_train['ActualElapsedTime'].fillna(x_train['ActualElapsedTime'].median(), inplace = True)
x_train['CRSElapsedTime'].fillna(x_train['CRSElapsedTime'].median(), inplace = True)
x_train['AirTime'].fillna(x_train['AirTime'].median(), inplace = True)
x_train['ArrDelay'].fillna(x_train['ArrDelay'].median(), inplace = True)
x_train['TaxiIn'].fillna(x_train['TaxiIn'].median(), inplace = True)
x_train['TaxiOut'].fillna(x_train['TaxiOut'].median(), inplace = True)
x_train['ArrTime'].fillna(x_train['ArrTime'].median(), inplace = True)


x_test['CarrierDelay'].fillna(x_test['CarrierDelay'].median(), inplace = True)
x_test['WeatherDelay'].fillna(x_test['WeatherDelay'].median(), inplace = True)
x_test['NASDelay'].fillna(x_test['NASDelay'].median(), inplace = True)
x_test['SecurityDelay'].fillna(x_test['SecurityDelay'].median(), inplace = True)
x_test['LateAircraftDelay'].fillna(x_test['LateAircraftDelay'].median(), inplace = True)
x_test['ActualElapsedTime'].fillna(x_test['ActualElapsedTime'].median(), inplace = True)
x_test['CRSElapsedTime'].fillna(x_test['CRSElapsedTime'].median(), inplace = True)
x_test['AirTime'].fillna(x_test['AirTime'].median(), inplace = True)
x_test['ArrDelay'].fillna(x_test['ArrDelay'].median(), inplace = True)
x_test['TaxiIn'].fillna(x_test['TaxiIn'].median(), inplace = True)
x_test['TaxiOut'].fillna(x_test['TaxiOut'].median(), inplace = True)
x_test['ArrTime'].fillna(x_test['ArrTime'].median(), inplace = True)


x_test.isna().sum()

In [ ]:
x_train[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'ArrDelay']].describe()

In [ ]:
x_train.isna().sum()

In [ ]:
x_train['Total_delay'] = x_train['CarrierDelay'] + x_train['WeatherDelay'] +\
                                     x_train['NASDelay'] + x_train['SecurityDelay'] +\
                                     x_train['LateAircraftDelay']


x_test['Total_delay'] = x_test['CarrierDelay'] + x_test['WeatherDelay'] +\
                                     x_test['NASDelay'] + x_test['SecurityDelay'] +\
                                     x_test['LateAircraftDelay']


x_train['Total_delay'].describe()

In [ ]:
x_test['Total_delay'].describe()

In [ ]:
x_train['Total_delay'].isna().sum() # проверка что все Nan изменены

In [ ]:
x_train['Total_delay'].hist()

In [ ]:
print(x_train['Total_delay'].shape)
dev = (x_train['Total_delay']-x_train['Total_delay'].mean()).abs()
# Стандартное отклонение
std = x_train['Total_delay'].std()
# Фильтруем исходный набор данных
x_train = x_train[~(dev>3*std)]
#правило трех сигм для отброса выборосов




In [ ]:
x_train['Total_delay'].describe()

In [ ]:
x_train['Total_delay'].isna().sum()

In [ ]:
print(x_train['Total_delay'].shape)
x_train['Total_delay'].hist()

In [ ]:
x_train.head()

In [ ]:
x_train.groupby('DayOfWeek')['Total_delay'].sum().plot()

In [ ]:
x_train.groupby('Cancelled')['Total_delay'].sum()
#после чистки остались только неотмененные рейсы, удаляем стоблец Cancelled

In [ ]:
x_train.drop('Cancelled', axis = 1, inplace = True)

x_test.drop('Cancelled', axis = 1, inplace = True)

In [ ]:
X = x_train.drop(['id', 'DelayTime'], axis = 1)
y = x_train['DelayTime']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
reg = Ridge(alpha = 1.0)
reg.fit(X_train, y_train)

In [ ]:
pred = reg.predict(X_test)

In [ ]:
mean_squared_error(pred, y_test)

In [ ]:
prediction = reg.predict(x_test)

In [ ]:
pd.Series(prediction, name='DelayTime').to_csv('prediction.csv', index_label='id', header=True)
test = pd.read_csv('prediction.csv')
test
#private MSE score - 7.30213 